# LangChain
### how chaining modules together can lead to advanced LLM features

In [2]:
import urllib.request

url = "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf"
file_name = "Phi-3-mini-4k-instruct-fp16.gguf"

urllib.request.urlretrieve(url, file_name)
print(f"File downloaded successfully as {file_name}")

File downloaded successfully as Phi-3-mini-4k-instruct-fp16.gguf


In [8]:
!pip install langchain langchain_community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
# .gguf is the compressed version of the model
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

llama_new_context_with_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32
llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_

In [10]:
# outputs empty string Phi-3 requires a specific prompt template
llm.invoke('Hello, My name is Mathias. What is 0 * 1?')

''

In [14]:
# There are many LLMs to choose from:
# https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/
from langchain import PromptTemplate
template = """<|user|>
{input_prompt}<|end|>
<|assistant>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [15]:
basic_chain = prompt | llm

In [16]:
basic_chain.invoke(
    {
        "input_prompt": "Hello, My name is Mathias. What is 1 * 0?",
    }
)

'|>The result of 1 * 0 is 0. In multiplication, any number multiplied by zero results in zero. This is because the operation of multiplication essentially means repeated addition and if you add a number (in this case, 1) to itself zero times it will always be zero. Hence, 1*0 equals 0.'

In [20]:
!pip install --upgrade langchain


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [28]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
#from langchain.chains import RunnableSequence

template = """<|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title_chain = title_prompt | llm
# title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")
title_chain.invoke({"summary": "a man that needed a ring to rule a kingdom"})

' "The Ring of Sovereigns: A Man\'s Quest for Kingdom Rule"'

In [29]:
template = """<|user|>
Describe the main character of a story about {summary} with the title {title}.
User only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(template=template, input_variables=["summary", "title"])
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [30]:
template = """<|user|>
Create a story about {summary} with the title {title}. The main character is:
{character}. Only return the story and it cannot be longer than one paragraph.
<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(template=template, input_variables=["summary", "title", "character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")
llm_chain = title | character | story

In [31]:
llm_chain.invoke("A lord that needed a ring to rule the kingdom")

{'summary': 'A lord that needed a ring to rule the kingdom',
 'title': ' "The Ring of Dominion: A Lords\' Quest for Power"',
 'character': ' The main character, Lord Aelar, is a noble and resourceful ruler burdened with the responsibility of maintaining peace in his kingdom. Driven by ambition and determination, he embarks on an epic quest to find the legendary Ring of Dominion, believing it holds the key to absolute power and prosperity for his realm.',
 'story': " In the heart of the enchanted kingdom of Eldoria, Lord Aelar, a noble and resourceful ruler, stood on the precipice of his destiny. Burdened with maintaining peace in his realm while yearning for greater power to usher in an era of prosperity, he resolved to embark upon an epic quest for The Ring of Dominion—a legendary artifact said to grant absolute control over all lands and peoples. Guided by ambition and unwavering determination, Lord Aelar traversed treacherous terrains, battled fierce creatures, and outwitted cunning

In [32]:
basic_chain.invoke({"input_prompt": "Hello, My name is Mathias. What is 1 * 0?"})

'|>Hello Mathias! In mathematics, when you multiply any number by zero, the result is always zero. So, 1 * 0 equals to 0. This rule holds true for all numbers, not just one.'

In [33]:
basic_chain.invoke({"input_prompt": "What is my name?"})

"|>I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I can't know your name without that information."

### ConversationBufferMemory

In [58]:
template = """<|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|> assistant is the Ai"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [59]:
from langchain.memory import ConversationBufferMemory
# from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [60]:
llm_chain.invoke({"input_prompt": "Hello, My name is Mathias. What is 1 * 0?"})

{'input_prompt': 'Hello, My name is Mathias. What is 1 * 0?',
 'chat_history': '',
 'text': ' giving the answer to the mathematical question.\n\nThe answer is: 0. When you multiply any number by zero, the result is always zero.'}

In [61]:
llm_chain.invoke({"input_prompt": "What is my name?"}) # added `assistant is the Ai` otherwise my name would be: 'assistant'

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hello, My name is Mathias. What is 1 * 0?\nAI:  giving the answer to the mathematical question.\n\nThe answer is: 0. When you multiply any number by zero, the result is always zero.',
 'text': ', and the answer to "What is my name?" based on the given conversation is Mathias.\n\nHowever, if you\'re referring to the AI itself, it doesn\'t have a personal name as it\'s an artificial intelligence system designed for interactions like this one. It can be referred to by its programmed name or identifier used in this particular instance.'}

In [62]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

/var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_10760/1814494710.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [64]:
llm_chain.predict(input_prompt="Hello! My name is Mathias and I have hazel eyes, what is 1 * 0?")

' responding to the mathematical question:\n\n"Hello Mathias! When you multiply 1 by 0, the result is always 0. How else can I assist you today?"'

In [65]:
llm_chain.predict(input_prompt="What is 6!?")

', 6 factorial (denoted as 6!) is the product of all positive integers from 1 to 6. So, it\'s calculated as:\n\n6! = 6 × 5 × 4 × 3 × 2 × 1 = 720\n\nYou can say: "Mathias, 6 factorial (6!) is equal to 720."'

In [67]:
llm_chain.invoke({'input_prompt': "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hello! My name is Mathias and I have hazel eyes, what is 1 * 0?\nAI:  responding to the mathematical question:\n\n"Hello Mathias! When you multiply 1 by 0, the result is always 0. How else can I assist you today?"\nHuman: What is 6!?\nAI: , 6 factorial (denoted as 6!) is the product of all positive integers from 1 to 6. So, it\'s calculated as:\n\n6! = 6 × 5 × 4 × 3 × 2 × 1 = 720\n\nYou can say: "Mathias, 6 factorial (6!) is equal to 720."',
 'text': ', and you can address it as AI.\nYour name is Mathias, based on the information provided in the current conversation.'}

In [69]:
llm_chain.invoke({"input_prompt": "What is my eye color?"}) # doesn't know because history has a limit

{'input_prompt': 'What is my eye color?',
 'chat_history': 'Human: What is 6!?\nAI: , 6 factorial (denoted as 6!) is the product of all positive integers from 1 to 6. So, it\'s calculated as:\n\n6! = 6 × 5 × 4 × 3 × 2 × 1 = 720\n\nYou can say: "Mathias, 6 factorial (6!) is equal to 720."\nHuman: What is my name?\nAI: , and you can address it as AI.\nYour name is Mathias, based on the information provided in the current conversation.',
 'text': " and there is no information provided about your eye color. I don't have access to personal data unless it has been shared with me during our conversation."}

### A way around retaining longer k history elements (long-term memory) we can summarize the history using another LLM (here we used same LLM, but we could leverage a smaller one)

In [70]:
summary_prompt_template = """<|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(input_variables=["new_lines", "summary"], template=summary_prompt_template)

In [71]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", prompt=summary_prompt)
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_10760/3671776721.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", prompt=summary_prompt)


In [74]:
llm_chain.invoke({"input_prompt": "Hello, My name is Mathias. What is 3 * 0?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' Mathias introduced themselves as "Mathias" and asked for the result of 3 multiplied by 0. The AI responded, explaining that the answer is zero due to the multiplication property of zero. Additionally, when queried about their name, the AI clarified it doesn\'t have personal information since there hasn\'t been any mention during this interaction.',
 'text': ', and it does not have a name in the same sense as humans do. It\'s referred to by the name of its creation or platform (e.g., "Microsoft\'s ChatGPT").'}

In [75]:
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' Mathias initiated the conversation, inquiring about the result of 3 multiplied by 0. The AI explained that any number multiplied by zero equals zero because of the multiplication property of zero. Additionally, when asked about their name, the AI clarified it doesn\'t have a personal identity in human terms and is known as "Microsoft\'s ChatGPT."',
 'text': '\'s response to the question about 3 multiplied by 0.\n\nThe first question you asked was: "What is 3 multiplied by 0?"'}

In [76]:
memory.load_memory_variables({})

{'chat_history': ' Mathias began the conversation by asking, "What is 3 multiplied by 0?" The AI responded that any number multiplied by zero equals zero due to the multiplication property of zero. When introduced with their designation, the AI identified itself as "Microsoft\'s ChatGPT," noting it lacks personal identity in human terms.'}

## Agents

In [78]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 15.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [102]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "redacted"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [103]:
react_template = """Anser the following questions as best as you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""
prompt = PromptTemplate(template=react_template, input_variables=["tools", "tool_names", "input", "agent_scratchpad"])

In [104]:
!pip install -U duckduckgo-search


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [105]:
!pip install numexpr


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [106]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
search_tool = Tool(name="duckduck", description="A web search engine for general queries", func=search.run)

tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [107]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [108]:
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 1.02 EUR for 1 USD."
    }
)
# EXCEEDED OpenAI QUOTA :(



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: redacted. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}